# Visualisation of Moroccan's Collaboation Network ( Medical Field )
Les informations courante sur les collaborations des éditeurs marocains sont extraites par la biblithèque "SCOPUS", après on va essayer de traiter les informations basées sur PUBMED parce qu'il offre du "metadata" différente à celle du "SCOPUS" 

In [313]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import itertools
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

data = []
scopusIDs = set()
authorPairs = []

with open("data//scopusAuthorsAll.json", "r", encoding='utf-8') as f:
    jsonAuthorsList = json.load(f)
        
dfAuth = json_normalize(jsonAuthorsList)
dfAuth = dfAuth.groupby(['authname','affname','affcountry']).size().reset_index()
del(dfAuth[0])

with open("data//scopus.json", "r", encoding='utf-8') as f:
    jsonDocsList = json.load(f)
        

for row in jsonDocsList:
    authlist = set()
    
    for auth in row['authors']:
        authlist.add(auth['authname'])
    
    authlist = sorted(authlist, key=str.lower)
    l = list(itertools.combinations(authlist, 2))
    for r in l:
        r = list(r)
        r.insert(0, row['title'])
        authorPairs.append(r)
        
df = pd.DataFrame(authorPairs, columns = ('title', 'from', 'to'))
#df = df.loc[(df['from'] == 'Brainin M.') & (df['to'] == 'Tanne D.')]
#df = df[df['from'].str.contains("Tanne")]

df['coll_count'] = df.groupby('from')['to'].transform(pd.Series.value_counts)
df = df.groupby(['from','to']).size().reset_index()
df = df.rename(columns={0 : 'coll_count'})
df = df.sort_values(['coll_count'], ascending=False)

#********************Filter the information to contains Morocco********************

df = df.merge(dfAuth, left_on='from', right_on='authname')
del(df['authname'])
df = df[['from', 'affname', 'affcountry', 'to', 'coll_count']]
df = df.merge(dfAuth, left_on='to', right_on='authname')
del(df['authname'])
df = df[['from', 'affname_x', 'affcountry_x', 'to','affname_y', 'affcountry_y', 'coll_count']]


df = df[df['affcountry_x'].str.contains("Morocco")]
df = df[df['affcountry_y'].str.contains("Morocco")]
df = df.head(300)
#************************************************************************************


G = nx.from_pandas_dataframe(df, 'to', 'from', 'coll_count')

print(nx.info(G))

degree_centrality = nx.degree_centrality(G)
#sorted(degree_centrality.items(), key=lambda x: x[1], reverse = True)

Name: 
Type: Graph
Number of nodes: 55
Number of edges: 124
Average degree:   4.5091


#### Création du Graphe : 
On a créé un graphe avec le package Networkx, et la visualisation avec "D3.js Force Layout". 

In [316]:
from networkx.readwrite import json_graph
from IPython.core.display import display, HTML
from string import Template

for ix,deg in G.degree().items():
    G.node[ix]['degree'] = deg

for ix,cent in nx.degree_centrality(G).items():
    G.node[ix]['cent'] = cent

G.nodes(data=True)[:5]

data = json_graph.node_link_data(G)
with open('graph.json', 'w') as f:
    json.dump(data, f, indent=4)


In [317]:
HTML('<script src="lib/d3.v3.min.js"></script>')

In [318]:
css_text ='''
.node {
  stroke: #fff;
  stroke-width: 1.5px;
}

.link {
  stroke: #999;
  stroke-opacity: .6;
}
.node text {
  font: 10px sans-serif;
}
'''

In [319]:
js_text_template = Template('''

var width = 960, height = 1000;

var color = d3.scale.category20();

var force = d3.layout.force()
    .charge(-300)
    .linkDistance(150)
    .gravity(0)
    .size([width, height]);


var svg = d3.select("#$cont").append("svg:svg")
    .attr("width", width)
    .attr("height", height)
    .call(d3.behavior.zoom().on("zoom", redraw))
    .on("dblclick.zoom", null)
    .append('g');

    
var data = $json_data;
            
var graph = {'nodes' : data.nodes, 'links' : data.links};



  force
      .nodes(graph.nodes)
      .links(graph.links)
      .friction(0.1)
      .on('start', start)
      .start();
      
//---

function start() {
    var ticksPerRender = 300;

    requestAnimationFrame(function render() {
      for (var i = 0; i < ticksPerRender; i++) {
        force.tick();
      }
      links
        .attr('x1', function(d) { return d.source.x; })
        .attr('y1', function(d) { return d.source.y; })
        .attr('x2', function(d) { return d.target.x; })
        .attr('y2', function(d) { return d.target.y; });
      nodes
        .attr('cx', function(d) { return d.x; })
        .attr('cy', function(d) { return d.y; });
      
      if (force.alpha() > 0) {
        requestAnimationFrame(render);
      }
    })
  }
//----
  
  var link = svg.selectAll(".link")
      .data(graph.links)
      .enter().append("line")
      .attr("class", "link")
      .style("stroke-width", function(d) { return Math.sqrt(d.value); })
      .attr("transform", function(d) { return "translate(" + d+5 + ")"; });

  var node = svg.selectAll("circle")
          .data(graph.nodes)
          .enter().append("circle")
          .attr("class", "node")
          .attr("r", function(d) { return (d.degree + 4); })
          .attr("transform", function(d) { return "translate(" + d + ")"; })
          .style("fill", function(d){return color(d.degree);})
          .call(force.drag)
          .on('dblclick', connectedNodes);

          
      
      node.append("title")
        .text(function (d) { return "Degree : "+d.degree; });
      
  var nodelabels = svg.selectAll(".nodelabel") 
       .data(graph.nodes)
       .enter()
       .append("text")
       .attr({"x":function(d){return d.x;},
              "y":function(d){return d.y;},
              "class":"nodelabel",
              "stroke":"gray"})
       .attr("transform", function(d) { return "translate(" + d + ")"; })
       .text(function(d){return d.id;});
  
  force.on("tick", function() {
    link.attr("x1", function(d) { return d.source.x; })
        .attr("y1", function(d) { return d.source.y; })
        .attr("x2", function(d) { return d.target.x; })
        .attr("y2", function(d) { return d.target.y; });

    node.attr("cx", function(d) { return d.x; })
        .attr("cy", function(d) { return d.y; });
    
    nodelabels.attr("x", function(d) { return d.x; }) 
              .attr("y", function(d) { return d.y; });
        
  });
  


  
 //----------------------------------
//Toggle stores whether the highlighting is on
var toggle = 0;

//Create an array logging what is connected to what
var linkedByIndex = {};

for (var i = 0; i < graph.nodes.length; i++) {
    linkedByIndex[i + "," + i] = 1;
};

graph.links.forEach(function (d) {
    linkedByIndex[d.source.index + "," + d.target.index] = 1;
});

//Looks up whether a pair of nodes are neighbours.
function neighboring(a, b) {
    return linkedByIndex[a.index + "," + b.index];
}

function connectedNodes() {
    if (toggle == 0) {
        //Reduce the opacity of all but the neighbouring nodes to 0.3.
        var d = d3.select(this).node().__data__;
        node.style("opacity", function (o) {
            return neighboring(d, o) | neighboring(o, d) ? 1 : 0.3;
        });
        
        d3.selectAll(".node").attr("r",function(d) { return (d.degree + 10); }).style("stroke","black");
        
        nodelabels.style("opacity", function (o) {
            return neighboring(d, o) | neighboring(o, d) ? 1 : 0.3;
        });
        
        //Reduce the opacity of all but the neighbouring edges to 0.8.
        link.style("opacity", function (o) {
            return d.index==o.source.index | d.index==o.target.index ? 1 : 0.8;
        });
        link.style("stroke", "black");
        
        //Increases the stroke width of the neighbouring edges.
        link.style("stroke-width", function (o) {
            return d.index==o.source.index | d.index==o.target.index ? 3 : 0.8;
        });
        
        //Reset the toggle.
        toggle = 1;
    } else {
    
        //Restore everything back to normal
        d3.selectAll(".node").attr("r",function(d) { return (d.degree + 4); }).style("stroke","white");
        d3.selectAll(".link").style("stroke","grey").style("stroke-width",1);
        d3.selectAll(".link").transition().duration(500).style("opacity", 1);
        d3.selectAll(".node").transition().duration(500).style("opacity", 1);
        d3.selectAll(".nodelabel").style("opacity", 1);
        
        toggle = 0;
    }
}

//---------------------------------
    function redraw() {
          svg.attr("transform",
              "translate(" + d3.event.translate + ")"
              + " scale(" + d3.event.scale + ")");
        } 

        var drag = force.stop().drag()
                .on("dragstart", function(d) {
                    d3.select(this).classed("fixed", d.fixed = true);
                    d3.event.sourceEvent.stopPropagation(); // to prevent pan functionality from 
                                                            //overriding node drag functionality.
                    // put any other 'dragstart' actions here
                });
''')

In [320]:
html_template = Template('''
<style> $css_text </style>
<div id="cont"></div>
<div id="texttip"></div>
<script> $js_text </script>
''')

js_text = js_text_template.safe_substitute({'json_data' : json.dumps(data),'cont': 'cont'})


#### Instructions : 
Une fois la visualisation est lancée, il faut essayer de déplacer le noeud au corner du l'écran pour pouvoir lancer l'animation est le graphe au totalité.

#### Remarques : 
Une remarque est que avec un nombre très élevé de noeuds, l'animation du graphe est trés lente mais on essaye de trouver des solutions.

In [321]:
HTML(html_template.safe_substitute({'css_text': css_text, 'js_text': js_text}))